# Performance Check

In [1]:
import timeit
import numpy as np
from npgeohash import npgeohash, npgeohash_jit

In [2]:
poi = np.array(
    [
        [35.69240645093, 139.7034750767164],
        [35.691255831981294, 139.69025228754268],
        [35.68307317410146, 139.71622562341963],
        [35.70071598380815, 139.69749333875686],
        [35.68997734701496, 139.6847427920536],
        [35.68115524225217, 139.68584469115146],
    ]
)
lat, lon = 35.68952987243547, 139.69953972279566

results = {}


def _t(stmt):
    t = timeit.Timer(stmt, globals=globals())
    n = 10000
    u = t.timeit(n) / n * 1e6
    print(f"{u:.2f} us")
    results[stmt] = u

The following function is jitted.

In [3]:
_t("npgeohash.encode_array(poi, 7)")
_t("npgeohash_jit.encode_array(poi, 7)")

50.13 us
9.24 us


In [4]:
_t("list(npgeohash.create_circle(lat, lon, 1000, 7))")
_t("list(npgeohash_jit.create_circle(lat, lon, 1000, 7))")

1198.29 us
275.56 us


In [5]:
poi_geohashes = npgeohash.encode_array(poi, 7)

_t("npgeohash.neighbors(poi_geohashes[0])")
_t("npgeohash_jit.neighbors(poi_geohashes[0])")

53.48 us
15.93 us


The following functions are actually not jitted.

In [6]:
_t("npgeohash.many_neighbors(poi_geohashes)")
_t("npgeohash_jit.many_neighbors(poi_geohashes)")

328.07 us
320.56 us


In [7]:
_t("npgeohash.isin(poi_geohashes, poi_geohashes)")
_t("npgeohash_jit.isin(poi_geohashes, poi_geohashes)")

36.67 us
31.47 us


In [8]:
_t("npgeohash.isin_circle(poi_geohashes, lat, lon, 1000, 7)")
_t("npgeohash_jit.isin_circle(poi_geohashes, lat, lon, 1000, 7)")

2258.23 us
2263.92 us


# Result

In [9]:
from IPython.core.display import Markdown

r_nojit = {}
r_jit = {}
for func, time in results.items():
    i = func.find("npgeohash")
    j = func.find(".", i)
    k = func.find("(", j)
    module_name = func[i:j]
    func_name = func[j + 1:k]
    if module_name == "npgeohash_jit":
        r_jit[func_name] = time
    else:
        r_nojit[func_name] = time
table = ["Function|npgeohash(w/ jit)|npgeohash_jit(wo/ jit)|Times", "---|---|---|---"]
for func_name in r_nojit.keys():
    table.append(f"{func_name}|{r_jit[func_name]:.2f}|{r_nojit[func_name]:.2f}|{r_nojit[func_name] / r_jit[func_name]:.2f}")
display(Markdown("\n".join(table)))
    

Function|npgeohash(w/ jit)|npgeohash_jit(wo/ jit)|Times
---|---|---|---
encode_array|9.24|50.13|5.43
create_circle|275.56|1198.29|4.35
neighbors|15.93|53.48|3.36
many_neighbors|320.56|328.07|1.02
isin|31.47|36.67|1.17
isin_circle|2263.92|2258.23|1.00